In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [11]:
today = today - timedelta(days=1)

In [59]:


homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
rankings = pd.read_excel(file_name)

last_season = pd.read_csv('data/2022-23.csv')

In [60]:
g_last_season = last_season.loc[last_season['position'] == 'G']
s_last_season= last_season.loc[last_season['position'] != 'G']


In [89]:
s_last_season

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,...,evenStrengthSavePercentage,shutout,shortHandedSavePercentage,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
0,1023,0,0,0,2,0,0,0,0,0,...,NaN,0,NaN,1.0,0.2,0.0,NaN,NaN,NaN,NaN
1,1261,0,1,2,1,0,0,0,13,25,...,NaN,0,NaN,1.0,2.3,0.0,NaN,NaN,NaN,NaN
2,558,0,0,2,2,0,0,0,0,0,...,NaN,0,NaN,1.0,0.9,0.0,NaN,NaN,NaN,NaN
3,1283,1,0,1,3,0,0,2,0,0,...,NaN,0,NaN,1.0,2.4,0.0,NaN,NaN,NaN,NaN
4,634,0,0,0,0,0,0,2,0,0,...,NaN,0,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49988,876,0,0,1,0,0,0,0,2,3,...,NaN,0,NaN,1.0,0.6,0.0,NaN,NaN,NaN,NaN
49989,955,1,0,2,0,0,0,0,7,17,...,NaN,0,NaN,1.0,1.2,0.0,NaN,NaN,NaN,NaN
49990,1570,0,0,3,0,0,0,0,0,0,...,NaN,0,NaN,1.0,0.8,0.0,NaN,NaN,NaN,NaN
49991,163,0,0,1,0,0,0,0,0,0,...,NaN,0,NaN,1.0,0.1,0.0,NaN,NaN,NaN,NaN


In [93]:
def summary_statistics(df):
    temp_build = df.groupby('id').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'timeOnIce': 'sum',
        'gamePlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
#         'blockedShots': 'sum',
#         'powerPlayToi': 'sum',
#         'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'powerPlayGoals': 'sum',
        'powerPlayAssists': 'sum',
        'fantasyPoints': 'sum'
#         'secondaryPosition': 'first',
#         'tertiaryPosition': 'first'
    }).reset_index()


    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['timeOnIce'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamePlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [94]:
last_season_sum = summary_statistics(s_last_season)

In [67]:
for index, row in player_bios.iterrows():
    try:
        player_bios.at[index, 'last_season'] = last_season_sum.loc[last_season_sum['id'] == row['playerId']]['FPPG'].iloc[0]
    except:
        player_bios.at[index, 'last_season'] = 0
player_bios

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last_season
0,8481559,J. Hughes,NJD,C,13377,11,40.0,10.76,3.64,0,...,240.24,Jack Hughes,2001-05-14,L,71,175,99.528697,C,4565222,2.66
1,8475660,C. Talbot,LAK,G,46333,13,57.4,4.46,4.42,0,...,234.68,Cam Talbot,1987-07-05,L,76,200,76.091815,G,5734,0.00
2,8477956,D. Pastrnak,BOS,R,18279,16,55.0,10.83,3.44,0,...,227.04,David Pastrnak,1996-05-25,R,72,196,99.906033,RW,3114778,3.16
3,8476453,N. Kucherov,TBL,R,21962,17,59.4,9.74,3.49,0,...,223.36,Nikita Kucherov,1993-06-17,L,71,182,99.870795,RW,2563060,2.62
4,8479318,A. Matthews,TOR,C,21586,17,57.6,9.61,3.39,0,...,220.35,Auston Matthews,1997-09-17,L,75,215,99.910438,C,4024123,3.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,8481035,S. Ersson,PHI,G,20969,6,-7.4,-1.27,-1.23,0,...,-28.33,Samuel Ersson,1999-10-20,L,75,194,1.465982,G,4587815,0.00
804,8479973,S. Skinner,EDM,G,41419,12,-7.4,-0.64,-0.62,0,...,-30.62,Stuart Skinner,1998-11-01,L,76,230,48.378748,G,4268767,0.00
805,8482821,A. Soderblom,CHI,G,23314,7,-7.8,-1.20,-1.11,0,...,-35.18,Arvid Soderblom,1999-08-19,L,75,180,1.049954,G,4894729,0.00
806,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,-35.58,Jack Campbell,1992-01-09,L,75,200,2.804288,G,5473,0.00


In [83]:
fade = player_bios.loc[(player_bios['FPPG'] <= 1.7) & (player_bios['last_season'] >= 1.7)]
fade['difference'] = fade['last_season'] - fade['FPPG']
fade.sort_values('difference', ascending=False, inplace=True)
fade

for index, row in fade.iterrows():
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(100 - row['roster_percent'], 2)}% available; {round(row['difference'], 2)} FPPG drop "
    f"[{row['FPPG']} FPPG this season, {row['last_season']} last season]):"
)
    print()

<p><b>Brandon Montour, D, Florida Panthers </b>(11.19% available; 2.22 FPPG drop [0.1 FPPG this season, 2.32 last season]):

<p><b>Lukas Rousek, R, Buffalo Sabres </b>(99.97% available; 1.56 FPPG drop [0.34 FPPG this season, 1.9 last season]):

<p><b>Tyson Foerster, R, Philadelphia Flyers </b>(98.51% available; 1.34 FPPG drop [0.94 FPPG this season, 2.28 last season]):

<p><b>Rafael Harvey-Pinard, L, Montréal Canadiens </b>(48.39% available; 1.28 FPPG drop [0.96 FPPG this season, 2.24 last season]):

<p><b>Jackson LaCombe, D, Anaheim Ducks </b>(99.78% available; 1.27 FPPG drop [1.08 FPPG this season, 2.35 last season]):

<p><b>Nico Hischier, C, New Jersey Devils </b>(15.5% available; 1.26 FPPG drop [0.99 FPPG this season, 2.25 last season]):

<p><b>Luke Hughes, D, New Jersey Devils </b>(20.75% available; 1.17 FPPG drop [1.23 FPPG this season, 2.4 last season]):

<p><b>Sam Bennett, C, Florida Panthers </b>(63.17% available; 1.14 FPPG drop [0.64 FPPG this season, 1.78 last season]):

<p>

In [108]:
last_season_sum.sort_values('fantasyPoints', ascending=False, inplace=True)
last_season_sum['rank'] = last_season_sum['fantasyPoints'].rank(ascending=False)
last_season_sum.loc[last_season_sum['name'] == 'Jared McCann']

,id,name,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,plusMinus,powerPlayGoals,powerPlayAssists,fantasyPoints,FPP60,FPPG,rank
382,8477955,Jared McCann,SEA,LW,77448,79.0,40,30,19.0,210,65,18,7,9,161.0,7.48,2.04,71.0


In [111]:
start = s_last_season.loc[s_last_season['date'] <= '2022-11-20']
last_season_sum_start = summary_statistics(start)
last_season_sum_start.sort_values('fantasyPoints', ascending=False, inplace=True)
last_season_sum_start['rank'] = last_season_sum_start['fantasyPoints'].rank(ascending=False)
last_season_sum_start.loc[last_season_sum_start['name'] == 'Andrei Kuzmenko']

,id,name,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,plusMinus,powerPlayGoals,powerPlayAssists,fantasyPoints,FPP60,FPPG,rank
750,8483808,Andrei Kuzmenko,VAN,LW,15335,17.0,7,6,5.0,40,0,2,4,1,30.5,7.16,1.79,154.0
